In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications import resnet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

import cv2
import matplotlib.pyplot as plt

2024-02-04 21:10:45.466681: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-04 21:10:45.502353: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-04 21:10:45.502393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-04 21:10:45.503101: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-04 21:10:45.508369: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-04 21:10:45.509211: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
for dirname, _, filenames in os.walk('./data'):
    print(dirname)

./data
./data/cell_images
./data/cell_images/cell_images
./data/cell_images/cell_images/Parasitized
./data/cell_images/cell_images/Uninfected
./data/Parasitized
./data/Uninfected


In [4]:
def data_prep(parasitized, uninfected):
    
    parasitized = ["./data/Parasitized" + '/' +  parasitize for parasitize in parasitized]
    uninfected = ["./data/Uninfected" + '/' +  uninfect for uninfect in uninfected]

    labels = len(parasitized) * ['parasitized'] + len(uninfected) * ['uninfected']
    data = parasitized + uninfected

    return pd.DataFrame({'Image_Path': data , 'Labels': labels})

In [5]:
df = data_prep(os.listdir("./data/Parasitized/"), os.listdir("./data/Uninfected/"))

In [6]:
print("Shape of the DataFrame:", df.shape)
print("\nHead of the DataFrame:")
print(df.head())
print("\nTail of the DataFrame:")
print(df.tail())

Shape of the DataFrame: (27558, 2)

Head of the DataFrame:
                                          Image_Path       Labels
0  ./data/Parasitized/C100P61ThinF_IMG_20150918_1...  parasitized
1  ./data/Parasitized/C100P61ThinF_IMG_20150918_1...  parasitized
2  ./data/Parasitized/C100P61ThinF_IMG_20150918_1...  parasitized
3  ./data/Parasitized/C100P61ThinF_IMG_20150918_1...  parasitized
4  ./data/Parasitized/C100P61ThinF_IMG_20150918_1...  parasitized

Tail of the DataFrame:
                                              Image_Path      Labels
27553  ./data/Uninfected/C99P60ThinF_IMG_20150918_142...  uninfected
27554  ./data/Uninfected/C99P60ThinF_IMG_20150918_142...  uninfected
27555  ./data/Uninfected/C99P60ThinF_IMG_20150918_142...  uninfected
27556  ./data/Uninfected/C99P60ThinF_IMG_20150918_142...  uninfected
27557  ./data/Uninfected/C99P60ThinF_IMG_20150918_142...  uninfected


In [7]:
# Function to load and preprocess images
def load_images(file_paths, target_size=(64, 64)):
    images = []
    for path in file_paths:
        img = cv2.imread(path)
        img = cv2.resize(img, target_size)
        images.append(img)
    return np.array(images)

# Load and preprocess images
# Assuming df is your DataFrame with 'Image_Path' and 'Labels' columns
# Make sure to replace df with your actual DataFrame variable
X = load_images(df['Image_Path'])
y = df['Labels']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the CNN model
model.fit(X_train, (y_train == 'parasitized').astype(int), epochs=5, batch_size=32, validation_split=0.2)

# Extract features from the CNN model
cnn_features_train = model.predict(X_train)
cnn_features_test = model.predict(X_test)

# Reshape features for Random Forest
cnn_features_train = cnn_features_train.reshape(cnn_features_train.shape[0], -1)
cnn_features_test = cnn_features_test.reshape(cnn_features_test.shape[0], -1)

# Build a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(cnn_features_train, (y_train == 'parasitized'))

# Predict using the hybrid model (Random Forest)
rf_predictions = rf_model.predict(cnn_features_test)

# Evaluate the hybrid model (Random Forest)
rf_accuracy = accuracy_score((y_test == 'parasitized'), rf_predictions)
print()
print(f"Accuracy of the hybrid model with Random Forest: {rf_accuracy*100}")

2024-02-04 21:11:01.121383: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 216711168 exceeds 10% of free system memory.


Epoch 1/5
552/552 [==============================] - ETA: 0s - loss: 11.2671 - accuracy: 0.6357

2024-02-04 21:12:14.536136: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 54190080 exceeds 10% of free system memory.


552/552 [==============================] - 77s 135ms/step - loss: 11.2671 - accuracy: 0.6357 - val_loss: 0.9448 - val_accuracy: 0.6889
Epoch 2/5
552/552 [==============================] - 79s 143ms/step - loss: 0.6069 - accuracy: 0.7692 - val_loss: 0.7655 - val_accuracy: 0.7261
Epoch 3/5
552/552 [==============================] - 78s 141ms/step - loss: 0.3982 - accuracy: 0.8392 - val_loss: 0.5166 - val_accuracy: 0.8190
Epoch 4/5
552/552 [==============================] - 75s 137ms/step - loss: 0.3119 - accuracy: 0.8833 - val_loss: 0.4844 - val_accuracy: 0.8159
Epoch 5/5
552/552 [==============================] - 71s 128ms/step - loss: 0.2405 - accuracy: 0.9123 - val_loss: 0.4245 - val_accuracy: 0.8281


2024-02-04 21:17:21.312731: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 270901248 exceeds 10% of free system memory.


 13/173 [=>............................] - ETA: 1s

2024-02-04 21:17:31.751695: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67731456 exceeds 10% of free system memory.


173/173 [==============================] - 1s 8ms/step

Accuracy of the hybrid model with Random Forest: 80.17053701015965
